# Projekt Data Science
Skład sekcji: Tobiasz Pilnicki, Tomasz Migała, Michał Skorus

## Wprowadzenie
Dostarczone informacje dotyczą sieci supermarketów w Ekwadorze. Na ich podstawie mieliśmy przewidzieć przyszłą sprzedaż tak, aby wiadomo było ile towaru należy zamawiać, tak aby jak najmniej towaru się marnowało i jak najwięcej osób miało możliwość kupienia wszystkiego czego w danym momencie potrzebuje. Do dyspozycji mieliśmy kilka zbiorów danych z informacjami np. o świętach i dniach wolnych, cenach ropy naftowej, informację o sklepach oraz transakcjach dane treningowe oraz dane testowe. Nasze rozwiązanie bazuje na rozwiązaniu użytkownika Howoo Jang.

Poprzez rozwiązanie powyższego problemu można zmniejszyć ilość marnotrawionej żywności, zwiększyć zyski, ponieważ większa ilość towaru nie będzie wyrzucana oraz obniżyć koszty. Celem projektu jest wyznaczenie takiej ilości zamawianego towaru, aby nie mieć go ani za mało ani za dużo.


In [ ]:
# importowanie bibliotek
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# importowanie plików
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# importowanie biblioteki do wykresów i wizualizacji statystyk
import matplotlib.pyplot as plt        
import seaborn as sns

In [ ]:
path = '/kaggle/input/store-sales-time-series-forecasting/'
os.listdir(path)

In [ ]:
data_oil = pd.read_csv(path+'oil.csv')
train_data = pd.read_csv(path+'train.csv', index_col=0)
test_data = pd.read_csv(path+'test.csv', index_col=0)
samp_subm = pd.read_csv(path+'sample_submission.csv')
data_holi = pd.read_csv(path+'holidays_events.csv')
data_store =  pd.read_csv(path+'stores.csv')
data_trans = pd.read_csv(path+'transactions.csv')

In [ ]:
print(f'Number of data_oil samples: {data_oil.shape}')
print(f'Number of train_data samples: {train_data.shape}')
print(f'Number of test_data samples: {test_data.shape}')
print(f'Number of samp_subm samples: {samp_subm.shape}')
print(f'Number of data_holi samples: {data_holi.shape}')
print(f'Number of data_store samples: {data_store.shape}')
print(f'Number of data_trans samples: {data_trans.shape}')
print(train_data.info())
print(train_data.columns)
print(train_data.head())

In [ ]:
print(data_trans.head())
print(train_data.head())

In [ ]:
print(train_data['store_nbr'].count())
print(train_data['store_nbr'].unique())
print(len(train_data['store_nbr'].unique()))

# Są 54 sklepy

In [ ]:
print(data_oil.head())

Ekwador jest zależy od kursu ceny ropy naftowej.

## Zależność ceny ropy od daty
Wykres ten przedstawia zależność ceny ropy od daty. Ceny w badanym kraju są mocno zależne od cen ropy dlatego wykres ten jest punktem zaczepienia w dalszych analizach. Jak można zauważyć wraz z kolejnymi datami cena ropy spada co teoretycznie sugeruje nam również możliwy spadek cen innych produktów.


In [ ]:
ax = data_oil.set_index('date').plot(figsize = (16, 8))
ax.set_xlabel('Date', fontsize = 'large')
ax.set_ylabel("Crude Oil", fontsize = 'large')

## Średnia liczba sprzedanych produktów/transakcji 
Na wykresach tych przedstawiona jest zależność średniej liczby sprzedanych produktów oraz średniej liczby transakcji w zależności od daty z nałożoną na nie tygodniową średnią liczbą sprzedaży. Na pierwszym wykrese możemy odnotować mocny wzrost z 200 produktami sprzedanymi w roku 2013 aż do nawet 600 sprzedanych w roku 2017. Widać tutaj zbieżność z wykresem ceny ropy naftowej - spadek ceny to większa liczba sprzedanych produktów. Natomiast liczba transakcji utrzymuje się na w miarę stałym poziomie. 


In [ ]:
# Przygotowanie danych, uśrednienie, pogrupowanie po dacie, wykorzystanie średniej ruchomej i wyznaczenie wykresu
avg_sales = train_data.groupby('date').agg({'sales': 'mean'}).reset_index()
avg_sales['weekly_avg_sales'] = avg_sales['sales'].ewm(span=7, adjust=False).mean()
ax1 = avg_sales.plot(x= 'date', y= ['sales', 'weekly_avg_sales'], figsize=(18,6))

avg_transactions = data_trans.groupby('date').agg({'transactions': 'mean'}).reset_index()
avg_transactions['weekly_avg_transactions'] = avg_transactions['transactions'].ewm(span=7, adjust=False).mean()
ax2 = avg_transactions.plot(x= 'date', y= ['transactions', 'weekly_avg_transactions'], figsize=(18,6))

In [ ]:
print(data_oil.head())
print(avg_sales.head())
print(avg_transactions.head())

## Wyznaczenie korelacji
Następnym krokiem było wyznaczenie korelacji pomiędzy ceną ropy a liczbą sprzedaży oraz transakcji. Jak widzimy w tabelce liczba sprzedaży jest ujemnie skorelowana z ceną paliwa co przy wzroście jednego oznacza spadek drugiego i na odwrót, co zgadzałoby się z poprzednimi wykresami. Natomiast liczba transakcji nie jest skorelowana z ceną ropy. 


In [ ]:
data_oil['sales'] = avg_sales['sales']
data_oil['transactions'] = avg_transactions['transactions']
#print(data_oil.head())
data_oil.corr()

Nie ma korelacji między ceną ropy, a transakcjami. Korelacja między ceną ropy a sprzedażą jest ujemnie skorelowana i nie ma większego wpływu. 

## Sprawdzenie najczęściej sprzedawanych kategorii względem ogólnej sprzedaży

In [ ]:
print(train_data.family.unique())
print(len(train_data.family.unique()))
train_data['family'] = train_data['family'].astype('category')
train_data['family_category'] = train_data['family'].cat.codes

family_category = dict( zip( train_data['family_category'], train_data['family'] ) )
family_category

# Są 33 kategorie produktów

Kolejny wykres przedstawia udział danej kategorii produktów w łącznej sprzedaży w celu sprawdzenia, które kategorie mają największy wpływ na sprzedaż spośród 33 kategorii. Jak widać artykuły spożywcze oraz napoje dominują sprzedaż osiągając łącznie ponad 50% sprzedawanych artykułów.

In [ ]:
# wykres udziału sprzedaży kategorii 

data_grouped_family_types = train_data.groupby(['family_category']).mean()[['sales', 'onpromotion']]

data_grouped_family_types['%_s'] = 100 * data_grouped_family_types['sales'] / data_grouped_family_types['sales'].sum()
data_grouped_family_types['%_s'] = data_grouped_family_types['%_s'].round(decimals = 3)


percent = 100 * data_grouped_family_types['sales'] / data_grouped_family_types['sales'].sum()
percent = percent.round(decimals = 3)
patches, texts = plt.pie(data_grouped_family_types['%_s'], startangle=90, radius=1.5)


lables_2 = ['{0} - {1:1.2f} %'.format(i,j) for i,j in zip(family_category.values(), percent)]


sort_legend = True
if sort_legend:
    patches, labels, dummy =  zip(*sorted(zip(patches, lables_2, data_grouped_family_types['%_s']),
                                          key=lambda x: x[2],
                                          reverse=True))
    
plt.legend(patches, labels, loc='best', bbox_to_anchor=(-0.1, 1.),
           fontsize=8)

Następny wykres przedstawia udział danej kategorii produktów w łącznej sprzedaży przy promocjach.

In [ ]:
# wykres udziału sprzedaży kategorii na promocji

data_grouped_family_types = train_data.groupby(['family_category']).mean()[['sales', 'onpromotion']]


data_grouped_family_types['%_p'] = 100 * data_grouped_family_types['onpromotion'] / data_grouped_family_types['onpromotion'].sum()
data_grouped_family_types['%_p'] = data_grouped_family_types['%_p'].round(decimals = 3)


percent = 100 * data_grouped_family_types['onpromotion'] / data_grouped_family_types['onpromotion'].sum()
percent = percent.round(decimals = 3)
patches, texts = plt.pie(data_grouped_family_types['%_p'], startangle=90, radius=1.5)


lables_2 = ['{0} - {1:1.2f} %'.format(i,j) for i,j in zip(family_category.values(), percent)]


sort_legend = True
if sort_legend:
    patches, labels, dummy =  zip(*sorted(zip(patches, lables_2, data_grouped_family_types['%_p']),
                                          key=lambda x: x[2],
                                          reverse=True))
    
plt.legend(patches, labels, loc='best', bbox_to_anchor=(-0.1, 1.),
           fontsize=8)

## Liczba sprzedanych produktów w różnych przedziałach czasowych.
Następnie wyznaczone zostały wykresy zależności sprzedaży od dnia tygodnia miesiąca oraz roku. Jak można zauważyć na wykresie zależności od dnia tygodnia, koniec tygodnia jest najbardziej oblegany jeżeli chodzi o sprzedaż. Jeżeli chodzi o miesiąc z największą liczbą sprzedanych produktów jest to koniec roku czyli grudzień co zapewne wiąże się z świętami oraz zabawą sylwestrową. Natomiast jeżeli chodzi o rok to z roku na rok widać wyraźny wzrost co sugeruje związek z malejącą ceną ropy.

In [ ]:
train_data['date'] = pd.to_datetime(train_data['date'])
train_data['day_of_week'] = train_data['date'].dt.dayofweek
train_data['month'] = train_data['date'].dt.month
train_data['year'] = train_data['date'].dt.year

In [ ]:
data_grouped_day = train_data.groupby(['day_of_week']).mean()['sales']
data_grouped_month = train_data.groupby(['month']).mean()['sales']
data_grouped_year = train_data.groupby(['year']).mean()['sales']

plt.subplots(3,1, figsize=(20,5))
plt.subplot(131)
plt.title('sales - day')
data_grouped_day.plot(kind='bar', stacked=True)
plt.subplot(132)
plt.title('sales - month')
data_grouped_month.plot(kind='bar', stacked=True)
plt.subplot(133)
plt.title('sales - year')
data_grouped_year.plot(kind='bar', stacked=True)

## Wnioski
 * W perspektywie dni tygodnia, sobota i niedziela prezentuje najlepszą sprzedaż.
 * W perspektywie miesięcy roku, grudniowa sprzedaż jest szczególnie wysoka.
 * W perspektywie lat, widać tendencję wzrostową.

## Liczba sprzedanych produktów w zależności od święta
Wykres przedstawiający liczbę sprzedanych produktów w zależności od typu święta, jak widać nie ma tutaj większego odchyłu, sprzedaż jest na poziomie zbliżonym jak do dni weekendowych. 

In [ ]:
print(data_holi['type'].unique())
print(data_holi['type'].value_counts())

day_type = data_holi[['date', 'type']]
avg_sales = train_data.groupby('date').agg({'sales': 'mean'}).reset_index()

day_type['date'] = pd.to_datetime(day_type['date'])
avg_sales['date'] = pd.to_datetime(avg_sales['date'])

#print(day_type.head())
#print(avg_sales.head())

df = pd.merge_asof(day_type, avg_sales, on = 'date')
df.dropna(inplace= True)
df.reset_index(drop = True, inplace= True)

#print(df.head())

df_1 = df.groupby(['type']).mean()['sales']
average_holiday_sales = df_1.mean()
#print(df_1.head())

print(f'average holiday sales is {average_holiday_sales}')

df_1.plot(kind='bar', figsize = (12,6)).set_title('average holiday sales')

## Średnia sprzedaż w dni świąteczne jest porównywalna do sprzedaży weekendowej

## Trend średniej liczby sprzedaży
Zgodnie z podpowiedzią jednego z moderatorów serwisu Kaggle zastosowany został model regresji liniowej i wyznaczona została linia trendu. Tutaj mamy dwa wykresy jeden reprezentujący regresję liniową średniej liczby sprzedaży, jak widać mamy tutaj do czynienia z tendencją wzrostową oraz drugi na którym zaprezentowana jest linia trendu wyznaczona przez model regresji liniowej. Jak widać wykresy te pokrywają się co oznacza poprawność przewidywania modelu. 

[https://www.kaggle.com/ryanholbrook/linear-regression-with-time-series/tutorial](https://www.kaggle.com/ryanholbrook/linear-regression-with-time-series/tutorial)

In [ ]:
avg_sales = train_data.groupby('date').agg({'sales': 'mean'}).reset_index()
avg_sales['Time'] = np.arange(len(avg_sales.index))
avg_sales.head()

In [ ]:
import seaborn as sns

plt.style.use("seaborn-whitegrid")
plt.rc(
    "figure",
    autolayout=True,
    figsize=(12, 6),
    titlesize=18,
    titleweight='bold',
)

plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=16,
    titlepad=10,
)

# Do użycia w opóźnionym o dzień wykresie
plot_params = dict(
    color = '0.75',
    style = ".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
    legend=False,
)

%config InlineBackend.figure_format = 'retina' # Można usunąć

fig, ax = plt.subplots()
ax.plot('Time', 'sales', data=avg_sales, color='0.75')
ax = sns.regplot(x='Time', y='sales', data=avg_sales, ci=None, scatter_kws=dict(color='0.25'))
ax.set_title('Time Plot of sales');

In [ ]:
avg_sales['Lag_1'] = avg_sales['sales'].shift(1)
avg_sales = avg_sales.reindex(columns = ['date','sales', 'Lag_1','Time'])
avg_sales.head()

Porównanie sprzedaży opóźnionej o dzień i rzeczywistej, wyznaczając linie trendu, wskazuje również że dane mają tendencje wzrostową.

In [ ]:
fig, ax = plt.subplots()
ax = sns.regplot(x = 'Lag_1', y = 'sales', data = avg_sales, ci = None, scatter_kws = dict(color='0.25'))
ax.set_aspect('equal')
ax.set_title('Lag Plot of sales')

In [ ]:
from sklearn.linear_model import LinearRegression

# Dane treningowe
X = avg_sales.loc[:, ['Time']] # features
y = avg_sales.loc[:, 'sales'] # target

# Trenowanie modelu
model = LinearRegression()
model.fit(X, y)

# Przypisanie przetrenowanych wartości jako szereg czasowy z takim samym indeksem czasowym jak dane treningowe
y_pred = pd.Series(model.predict(X), index = X.index)
y_pred

In [ ]:
ax = y.plot(**plot_params)
ax = y_pred.plot(ax=ax, linewidth = 3)
ax.set_title('Time Plot of sales');

In [ ]:
from sklearn.linear_model import LinearRegression

X = avg_sales.loc[:, ['Lag_1']] # Zdefiniowanie zbioru treningowego i jego atrybutów
X.dropna(inplace = True) # Usunięcie wartości brakujących w zbiorze treningowym
y = avg_sales.loc[:, 'sales'] # Zdefiniownie celu treningu
y, X = y.align(X, join = 'inner') # Usunięcie wartości przy brakujących danych

# Utworzenie instancji modelu regresji liniowej
model = LinearRegression()

# Przetrenowanie modelu
model.fit(X, y)

y_pred = pd.Series(model.predict(X), index=X.index)
y_pred

In [ ]:
fig, ax = plt.subplots()
ax.plot(X['Lag_1'], y, '.', color='0.25')
ax.plot(X['Lag_1'], y_pred)
ax.set_aspect('equal')
ax.set_ylabel('sales')
ax.set_xlabel('Lag_1')
ax.set_title('Lag Plot of sales');

## Lagged variable regression
Utworzony został model, który został wytrenowany na podstawie danych przesuniętych o jeden dzień w tył. Przewidziane wyniki zostały nałożone na dane oryginalne, jak widać przewidywane wartości są zbliżone.


In [ ]:
ax = y.plot(**plot_params)
ax = y_pred.plot()

## Dzięki regresji liniowej można wygładzić dane.

## Trend na podstawie średniej kroczącej
Jedną z metod wygładzających jest średnia krocząca i pozwoliła ona wygładzić zachowanie średniej liczby sprzedaży. Na wykresie średnia ta jest zaprezentowana, jak widać podąża za linią trendu.


In [ ]:
from pathlib import Path
from warnings import simplefilter

simplefilter("ignore") # ignorowanie ostrzeżeń, żeby wyczyścić komórki wyjściowe.

plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True, figsize=(11, 5))
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
    legend=False,
)
%config InlineBackend.figure_format = 'retina'

# Załadowanie danych o sprzedaży
avg_sales = train_data.groupby('date').agg({'sales': 'mean'}).reset_index()
#avg_sales = avg_sales.set_index('date')
#avg_sales.index = pd.to_datetime(avg_sales.index)
avg_sales = avg_sales.set_index('date').to_period("D")
avg_sales.head()

In [ ]:
moving_average = avg_sales.rolling(
    window=365,       # ustawienie okna na 365 dni
    center=True,      # ustawienie średniej w środku okna
    min_periods=183,  # minimalny ilość okresów jako około połowa rozmiaru okna
).mean()              # obliczenie średniej

ax = avg_sales.plot(style=".", color="0.5")
moving_average.plot(
    ax=ax, linewidth=3, title="sales - 365-Day Moving Average", legend=False,
);

In [ ]:
from statsmodels.tsa.deterministic import DeterministicProcess

dp = DeterministicProcess(
    index=avg_sales.index,  # daty z danych treningowych
    constant=True,       # dummy atrybut dla wskaźnika bias (y-intercept)
    order=1,             # trend (order 1 znaczy liniowy)
    drop=True,           # uniknięcie kolinearności
)
# `in_sample` tworzy cechy dla danych dat w argumencie `index`
X = dp.in_sample()

X.head()

In [ ]:
from sklearn.linear_model import LinearRegression
y = avg_sales["sales"]  # cel treningu

model = LinearRegression(fit_intercept=False)
model.fit(X, y)

y_pred = pd.Series(model.predict(X), index=X.index)
y_pred.tail()

## Przewidziany przez model trend 
Wykres wyznaczony przez model regresji liniowej przedstawiający trend bazujący na wyznaczonej wcześniej średniej kroczącej.


In [ ]:
ax = avg_sales.plot(style=".", color="0.5", title="sales - Linear Trend")
_ = y_pred.plot(ax=ax, linewidth=3, label="Trend")

In [ ]:
X = dp.out_of_sample(steps=180)

y_fore = pd.Series(model.predict(X), index=X.index)

y_fore.head()

## Przewidziany przez model wzrost
Wykres przedstawia przewidziany przez model trend sprzedaży w przyszłości, który wskazuje na dalsze wzrosty.


In [ ]:
ax = avg_sales["2013-01":].plot(title="Tunnel Traffic - Linear Trend Forecast", **plot_params)
ax = y_pred["2013-01":].plot(ax=ax, linewidth=3, label="Trend")
ax = y_fore.plot(ax=ax, linewidth=3, label="Trend Forecast", color="C3")
_ = ax.legend()

## Sezonowość

In [ ]:
# Zdefiniowanie funkcji do wyznaczania wykresów związanych z sezonowością

from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

# ustawienie domyślnych wartości biblioteki do wykresów
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True, figsize=(11, 5))
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=16,
    titlepad=10,
)
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
    legend=False,
)
%config InlineBackend.figure_format = 'retina'


# odniesienie: https://stackoverflow.com/a/49238256/5769929
def seasonal_plot(X, y, period, freq, ax=None):
    if ax is None:
        _, ax = plt.subplots()
    palette = sns.color_palette("husl", n_colors=X[period].nunique(),)
    ax = sns.lineplot(
        x=freq,
        y=y,
        hue=period,
        data=X,
        ci=False,
        ax=ax,
        palette=palette,
        legend=False,
    )
    ax.set_title(f"Seasonal Plot ({period}/{freq})")
    for line, name in zip(ax.lines, X[period].unique()):
        y_ = line.get_ydata()[-1]
        ax.annotate(
            name,
            xy=(1, y_),
            xytext=(6, 0),
            color=line.get_color(),
            xycoords=ax.get_yaxis_transform(),
            textcoords="offset points",
            size=14,
            va="center",
        )
    return ax


def plot_periodogram(ts, detrend='linear', ax=None):
    from scipy.signal import periodogram
    fs = pd.Timedelta("1Y") / pd.Timedelta("1D")
    freqencies, spectrum = periodogram(
        ts,
        fs=fs,
        detrend=detrend,
        window="boxcar",
        scaling='spectrum',
    )
    if ax is None:
        _, ax = plt.subplots()
    ax.step(freqencies, spectrum, color="purple")
    ax.set_xscale("log")
    ax.set_xticks([1, 2, 4, 6, 12, 26, 52, 104])
    ax.set_xticklabels(
        [
            "Annual (1)",
            "Semiannual (2)",
            "Quarterly (4)",
            "Bimonthly (6)",
            "Monthly (12)",
            "Biweekly (26)",
            "Weekly (52)",
            "Semiweekly (104)",
        ],
        rotation=30,
    )
    ax.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
    ax.set_ylabel("Variance")
    ax.set_title("Periodogram")
    return ax

# Załadownie danych o sprzedaży
avg_sales = train_data.groupby('date').agg({'sales': 'mean'}).reset_index()
avg_sales = avg_sales.set_index('date').to_period("D")
avg_sales.head()

## Sezonowość (dni w tygodniu)
Następnie zbadana została sezonowość danych. Mamy więc przedstawiony wykres zależności liczby sprzedaży od dni tygodnia dla każdego z tygodni roku, jak widać większość z tygodni zachowuje się bardzo podobnie, ze wzrostem pod koniec tygodnia oraz ze spadkiem w środku, z czego wynika, że największe zapotrzebowanie na produkty jest weekendami.

## Sezonowość (dni w roku)
Zbadano sezonowość względem lat
Tutaj mamy wykres zależności roku od dnia w roku, jak widać z dominacją roku 2017, jednak tylko do połowy ze względu na brak dalszych danych. 



In [ ]:
X = avg_sales.copy()

X['day'] = X.index.dayofweek # Dodanie kolumny dzień tygodnia
X['week'] = X.index.week # Dodanie kolumny z numerem tygodnia

X['dayofyear'] = X.index.dayofyear # Dodanie kolumny dzień roku
X['year'] = X.index.year # Dodanie kolumny z rokiem

fig, (ax0, ax1) = plt.subplots(2, 1, figsize=(11, 12))
seasonal_plot(X, y="sales", period="week", freq="day", ax=ax0)
seasonal_plot(X, y="sales", period="year", freq="dayofyear", ax=ax1);

## Periodogram
Na wykresach tych mamy przedstawioną wariancję liczby sprzedaży w zależności od danego okresu, jak możemy zauważyć największa wariancja występuje z tygodnia na tydzień, wtedy możemy też spodziewać się największych zmian. Dla reszty okresów wariancja ma wartość pomijalną.


In [ ]:
# plot_periodogram(avg_sales.sales);

y_deseason = y - y_pred

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, sharey=True, figsize=(10, 7))
ax1 = plot_periodogram(y, ax=ax1)
ax1.set_title("Product Sales Frequency Components")
ax2 = plot_periodogram(y_deseason, ax=ax2);
ax2.set_title("Deseasonalized");

In [ ]:
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

fourier = CalendarFourier(freq="A", order=10)  # 10 sin/cos par krzywych dla rocznej sezonowości

dp = DeterministicProcess(
    index=avg_sales.index,
    constant=True,   # dummy atrybut dla wskaźnika bias (y-intercept)
    order=1,         # trend (order 1 znaczy liniowy)
    seasonal=True,   # tygodniowa sezonowość (indicators)
    additional_terms=[fourier], # roczna sezonowość
    drop=True,       # uniknięcie kolinearności
)

X = dp.in_sample() # tworzy zbiór treningowy z procesu deterministycznego
# X.head()

## Przewidywana sprzedaż
Wykres przedstawia średnią liczbę sprzedaży wraz z przewidzianą przez model liczbą sprzedaży w 2018 roku, można zauważyć tutaj pewną okresowość. Widać tutaj lekką rozbieżność pomiędzy faktycznymi danymi a przewidzianymi natomiast występuje tutaj logiczny wzrost w okresie świątecznym.


In [ ]:
y = avg_sales["sales"]

model = LinearRegression(fit_intercept=False)
_ = model.fit(X, y)

y_pred = pd.Series(model.predict(X), index=y.index)
X_fore = dp.out_of_sample(steps=180)
y_fore = pd.Series(model.predict(X_fore), index=X_fore.index)

ax = y.plot(color='0.25', style='.', title="sales - Seasonal Forecast")
ax = y_pred.plot(ax=ax, label="Seasonal")
ax = y_fore.plot(ax=ax, label="Seasonal Forecast", color='C3')
_ = ax.legend()

## Wypisanie świąt dostępnych z zbiorze holidays

In [ ]:
comp_dir = Path('../input/store-sales-time-series-forecasting')

holidays_events = pd.read_csv(
    comp_dir / "holidays_events.csv",
    dtype={
        'type': 'category',
        'locale': 'category',
        'locale_name': 'category',
        'description': 'category',
        'transferred': 'bool',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)
holidays_events = holidays_events.set_index('date').to_period('D')

# Państwowe i regionalne święta w zbiorze treningowym
holidays = (
    holidays_events
    .query("locale in ['National', 'Regional']")
    .loc['2017':'2017-08-15', ['description']]
    .assign(description=lambda x: x.description.cat.remove_unused_categories())
)

display(holidays)

## Nałożenie dni świątecznych na wykres sprzedaży
Średnia liczba sprzedaży z nałożonymi w 2017 roku świętami, jak widać największe odchylania od normy występują właśnie w święta jednak nie zawsze, więc nie jesteśmy w stanie stwierdzić poza faktem konkretnej daty czy mamy do czynienia ze świętem czy też nie. 


In [ ]:
ax = y_deseason.plot(**plot_params)
plt.plot_date(holidays.index, y_deseason[holidays.index], color='C3')
ax.set_title('National and Regional Holidays');

In [ ]:
X_holidays = pd.get_dummies(holidays) # zapisanie święta jako wartości 1/0 gdzie 1 oznacza święto

#print(X_holidays.head(14))

# dołączenie świąt do zbiory treningowego
X2 = X.join(X_holidays, on='date').fillna(0.0)

## Wynik modelu z uwzględenieniem świąt
Przewidywana średnia liczba sprzedaży przez model wraz z uwzględnieniem odchyleń w święta jak widać w 2017 roku przy uwzględnieniu danych o świętach model poprawnie przewiduje odpowiednie wzrosty spadki w średniej liczbie sprzedaży związanej właśnie ze świętami.


In [ ]:
model = LinearRegression().fit(X2, y)
y_pred = pd.Series(
    model.predict(X2),
    index=X2.index,
    name='Fitted',
)

y_pred = pd.Series(model.predict(X2), index=X2.index)
ax = y.plot(**plot_params, alpha=0.5, title="Average Sales", ylabel="items sold")
ax = y_pred.plot(ax=ax, label="Seasonal")
ax.legend();

## Model użyty do stworzenia pliku odpowiedzi
Wyłączając styczeń z 2017 roku uzyskaliśmy lepszy wynik. Rząd kalendarzu Fouriera został ustawiony na 4 a częstotliwość na miesięczną.

In [ ]:
store_sales = pd.read_csv(
    comp_dir / 'train.csv',
    usecols=['store_nbr', 'family', 'date', 'sales'],
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'sales': 'float32',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)
store_sales['date'] = store_sales.date.dt.to_period('D')
store_sales = store_sales.set_index(['store_nbr', 'family', 'date']).sort_index()

# zmieniliśmy okres pomijając styczeń 2017 w którym były wzrosty spowodowane okresem poświątecznym
# funkcja unstack przygotowuje dane treningowe do osiągnięcia pożądanych wyników
y = store_sales.unstack(['store_nbr', 'family']).loc["02-2017":"10-2017"]

# testowaliśmy różne paramatry dla cech fouriera
fourier = CalendarFourier(freq='M', order=4) # 7 - dobrze też działało
dp = DeterministicProcess(
    index=y.index,
    constant=True,
    order=1,
    seasonal=True,
    additional_terms=[fourier],
    drop=True,
)
X = dp.in_sample()

model = LinearRegression(fit_intercept=False)
model.fit(X, y)
y_pred = pd.DataFrame(model.predict(X), index=X.index, columns=y.columns)

In [ ]:
STORE_NBR = '1'  # 1 - 54
FAMILY = 'MAGAZINES'
# Aby zobaczyć listę kategorii odkomentuj
# display(store_sales.index.get_level_values('family').unique())

# wyświetlenie przykładowego wyniku dla jednego sklepu i rodziny
ax = y.loc(axis=1)['sales', STORE_NBR, FAMILY].plot(**plot_params)
ax = y_pred.loc(axis=1)['sales', STORE_NBR, FAMILY].plot(ax=ax)
ax.set_title(f'{FAMILY} Sales at Store {STORE_NBR}');

In [ ]:
df_test = pd.read_csv(
    comp_dir / 'test.csv',
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'onpromotion': 'uint32',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)
df_test['date'] = df_test.date.dt.to_period('D')
df_test = df_test.set_index(['store_nbr', 'family', 'date']).sort_index()

# stworzenie atrybutów dla zbiory testowego 16 dni do przodu
X_test = dp.out_of_sample(steps=16)
X_test.index.name = 'date'

y_submit = pd.DataFrame(model.predict(X_test), index=X_test.index, columns=y.columns)

y_submit = y_submit.stack(['store_nbr', 'family'])
y_submit = y_submit.join(df_test.id).reindex(columns=['id', 'sales'])
y_submit.to_csv('submission.csv', index=False)


https://www.kaggle.com/kashishrastogi/store-sales-forecasting

https://www.kaggle.com/shivamb/store-sales-forecasting-exploration

https://www.kaggle.com/drcapa/storesales-ts-starter

https://www.kaggle.com/kalilurrahman/store-sales-eda-prediction-with-ts

https://www.kaggle.com/shrutisaxena/store-sales-eda-using-plotly

https://www.kaggle.com/veleirx/store-sales-fast-eda#2.-Stores 

https://www.kaggle.com/learn/time-series